In [1]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [4]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [3]:
model = ChatOpenAI(temperature=0)

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

## JsonOutputFunctionsParser

In [6]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [8]:
parser = JsonOutputFunctionsParser()

In [9]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [10]:
chain.invoke({"input": "tell me a joke"})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

## JsonKeyOutputFunctionsParser

In [11]:
from typing import List

from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [12]:
class Jokes(BaseModel):
    """Jokes to tell user."""

    joke: List[Joke]
    funniness_level: int

In [13]:
parser = JsonKeyOutputFunctionsParser(key_name="joke")

In [14]:
openai_functions = [convert_pydantic_to_openai_function(Jokes)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [15]:
chain.invoke({"input": "tell me two jokes"})

[{'setup': "Why couldn't the bicycle stand up by itself?",
  'punchline': 'It was two tired.'},
 {'setup': 'What do you call a fish wearing a crown?',
  'punchline': 'A kingfish.'}]

## PydanticOutputFunctionsParser

In [16]:
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser

In [17]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

In [18]:
openai_functions = [convert_pydantic_to_openai_function(Joke)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [19]:
chain.invoke({"input": "tell me a joke"})

Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='It was two tired!')